In [35]:
import sys, os
sys.path.append(os.path.abspath(".."))
from data_process.data_processor import DataProcessor
from nuplan.common.actor_state.state_representation import Point2D
from nuplan.planning.scenario_builder.nuplan_db.nuplan_scenario_utils import ScenarioMapping
from nuplan.planning.scenario_builder.nuplan_db.nuplan_scenario_builder import NuPlanScenarioBuilder
from nuplan.planning.scenario_builder.scenario_filter import ScenarioFilter
from nuplan.planning.utils.multithreading.worker_parallel import SingleMachineParallelExecutor
from diffusion_planner.data_process.utils import (
get_scenario_map,
get_filter_parameters,
sampled_tracked_objects_to_tensor,
)

from diffusion_planner.data_process.roadblock_utils import (
  route_roadblock_correction
)


from diffusion_planner.data_process.agent_process import(sampled_tracked_objects_to_array_list,
                                                         sampled_static_objects_to_array_list,
                                                         agent_past_process,)
from diffusion_planner.data_process.map_process import(map_process,
                                                       get_neighbor_vector_set_map)

import numpy as np
import torch


In [2]:
db_path = "/cailiu2/Diffusion-Planner/data/2021.10.21.14.43.30_veh-28_01244_01519.db" # single db file
db_path = "/share/data_cold/open_data/nuplan/data/cache/mini"  # mini db files

map_path = "/share/data_cold/open_data/nuplan/maps"
save_path = "/cailiu2/Diffusion-Planner/data/processed"
map_version = "nuplan-maps-v1.0"

save_processed_path = "/cailiu2/Diffusion-Planner/data/processed"
total_scenarios = 10
device = torch.device("cuda:0")
scenario_mapping = ScenarioMapping(scenario_map=get_scenario_map(), subsample_ratio_override=0.5)
builder = NuPlanScenarioBuilder(db_path, map_path, None, None, map_version, scenario_mapping = scenario_mapping)
worker = SingleMachineParallelExecutor(use_process_pool=True)
scenario_filter = ScenarioFilter(*get_filter_parameters(num_scenarios_per_type=30000,
                                                          limit_total_scenarios=total_scenarios))
scenarios = builder.get_scenarios(scenario_filter, worker)
del worker, builder, scenario_filter
processor = DataProcessor(scenarios, device)

Worker threads: 128


In [3]:
from nuplan.planning.training.preprocessing.utils.agents_preprocessing import(sampled_past_ego_states_to_tensor)
"""
ego past
"""
processor.scenario = scenarios[0]
processor.anchor_ego_state = processor.scenario.initial_ego_state

past_ego_states = processor.scenario.get_ego_past_trajectory(
    iteration=0, num_samples=processor.num_past_poses, time_horizon=processor.past_time_horizon
)

sampled_past_ego_states = list(past_ego_states) + [processor.anchor_ego_state]
past_ego_states_tensor = sampled_past_ego_states_to_tensor(sampled_past_ego_states)
print(past_ego_states_tensor.shape)



torch.Size([21, 7])


In [10]:
processor = DataProcessor(scenarios, device)
processor.scenario = scenarios[0]
"""
ego future
"""
future_ego_states = processor.scenario.get_ego_future_trajectory(iteration=0, num_samples=processor.num_future_poses, time_horizon= processor.future_time_horizon)
future_ego_states_tensor = sampled_past_ego_states_to_tensor(list(future_ego_states))
print(future_ego_states_tensor.shape)
print(future_ego_states_tensor[0])
print(past_ego_states_tensor[-1])
print(past_ego_states_tensor[-2])
print(past_ego_states_tensor[-3])

torch.Size([50, 7])
tensor([ 3.3096e+05,  4.6911e+06,  5.4388e-01, -2.7254e-03,  7.0212e-04,
        -4.4483e-02,  1.1129e-02])
tensor([ 3.3096e+05,  4.6911e+06,  5.4387e-01, -2.3384e-03,  2.3940e-05,
         1.2597e-02,  1.2770e-02])
tensor([ 3.3096e+05,  4.6911e+06,  5.4387e-01, -6.4551e-03, -5.3419e-04,
        -1.2633e-02, -3.7846e-03])
tensor([ 3.3096e+05,  4.6911e+06,  5.4395e-01, -7.3130e-03,  5.7190e-05,
        -7.4196e-02, -8.6003e-03])


In [ ]:
neighbors = processor.scenario.initial_tracked_objects
neighbor_past = list(processor.scenario.get_past_tracked_objects(iteration= 0, time_horizon=2.0, num_samples = 10 * 2))
track_objs = neighbor_past[0].tracked_objects # center, track_token
print(len(track_objs))

neighbor_future = list(processor.scenario.get_future_tracked_objects(iteration= 0, time_horizon=5.0, num_samples = 10 * 5))

59


In [39]:
def process_tracked_objects(tracked_objects_in_all_frames, full_time_horizon: int):

  # 创建一个字典来存储所有障碍物的轨迹
  object_trajectories = {}

  # 遍历每一帧的 tracked_objects
  for frame_idx, detection_frame in enumerate(tracked_objects_in_all_frames):
      for obj in detection_frame.tracked_objects:
          obj_token = obj.track_token  # 获取障碍物 ID
          obj_position = np.array([obj.center.x, obj.center.y])  # 获取障碍物位置 (x, y)

          # 如果该障碍物 ID 还未存储，则初始化
          if obj_token not in object_trajectories:
              object_trajectories[obj_token] = []

          # 记录该障碍物在该帧的位置信息
          object_trajectories[obj_token].append(obj_position)

  # 统一轨迹长度，填充 NaN
  for obj_token, traj in object_trajectories.items():
      traj = np.array(traj)  # 转换为 NumPy 数组，形状为 (N, 2)
      num_frames = traj.shape[0]

      if num_frames < full_time_horizon:
          # 如果轨迹长度小于 V，则用 NaN 填充
          pad_size = full_time_horizon - num_frames
          pad_array = np.full((pad_size, 2), np.nan)  # 生成 (pad_size, 2) 的 NaN 数组
          object_trajectories[obj_token] = np.vstack([traj, pad_array])  # 拼接填充
      else:
          # 如果轨迹长度大于 V，则裁剪
          object_trajectories[obj_token] = traj[:full_time_horizon]

  return object_trajectories


In [40]:
object_traj = process_tracked_objects(neighbor_past, 50)

In [5]:
import os

# 处理时间格式

save_path = "/cailiu2/Diffusion-Planner/data/processed"
# 生成文件名
file_name = f"{processor.scenario.log_name}_{processor.scenario.token}_{processor.scenario.scenario_type}.pt"

# 确保文件名合法（移除特殊字符）
file_name = file_name.replace(":", "_").replace("/", "_").replace(" ", "_")

# 指定存放路径
os.makedirs(save_path, exist_ok=True)  # 确保目录存在

# 完整文件路径
file_path = os.path.join(save_path, file_name)
torch.save(data, file_path)


NameError: name 'data' is not defined

In [ ]:
import torch
print(torch.cuda.device_count())  # 如果为 True，表示有 CUDA 可用；如果为 False，则没有 CUDA 可用
print(torch.__version__)

2
1.12.1+cu113


/root/miniconda3/envs/dlp/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
